# Lsass Memory Dump via Comsvcs.dll

https://securitydatasets.com/notebooks/atomic/windows/credential_access/SDWIN-201018195009.html

https://attack.mitre.org/techniques/T1003/001

Attacker sometimes use built-in tools in Windows to dump credential materials from the memory of the Local Security Authority Subsystem Service (LSASS).  This hunt will look at one such technique, using the `MiniDump` exported function from `Comsvcs.dll`.

The dataset used here for demonstration is available from the SecurityDatasets.com page linked above. The *Simulation Metadata* section of that page shows the tool type as "manual" meaning the simulated adversary manually ran this command from a PowerShell session.

Well start by searching for the use of `rundll32.exe` to load `comsvcs.dll` and run the `MiniDump` function therein:

In [1]:
procs = GET process
        FROM https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/credential_access/host/psh_lsass_memory_dump_comsvcs.zip
        WHERE binary_ref.name = 'rundll32.exe'
              AND command_line LIKE '%comsvcs.dll% MiniDump%'

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),artifact*,directory*,file*,process*,user-account*,x-oca-asset*,x-oca-event*
procs,process,2,34,34,38,67,35,3,34,34


In [13]:
DISP procs ATTR binary_ref.name, command_line, pid

binary_ref.name,command_line,pid
rundll32.exe,"""C:\Windows\System32\rundll32.exe"" C:\windows\System32\comsvcs.dll MiniDump 756 C:\Users\wardog\AppData\Local\Temp\lsass-comsvcs.dmp full",4824


We found 1 instance; whether a given instance is malicious or not could depend on the related entities: the `process` that created this `process`, the associated `user-account`, etc.  Let's look at the parent process:

In [3]:
parents = FIND process CREATED procs

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),artifact*,directory*,file*,process*,user-account*,windows-registry-key*,x-oca-asset*,x-oca-event*
parents,process,2,29,60,66,95,62,4,24,60,60


In [6]:
DISP parents ATTR binary_ref.name, command_line, parent_ref.pid

binary_ref.name,command_line,parent_ref.pid
powershell.exe,C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe,None
powershell.exe,None,None


It's PowerShell, which is standard Windows shell.  It's also a popular tool for attackers, however.  The dataset we're using doesn't show anything beyond this, but if you re-run this hunt against a live data source you could continue following the trail here by `FIND`ing what process created powershell, etc.

Knowing the user will help round out the picture for the incident report.

In [10]:
users = FIND user-account OWNED procs

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),artifact*,directory*,file*,process*,user-account*,windows-registry-key*,x-oca-asset*,x-oca-event*
users,user-account,2,32,88,91,120,89,30,24,88,88


In [11]:
DISP users

user_id,id
WORKSTATION5\wardog,user-account--c8b6cbe8-e16c-5b03-948a-085d1333e43e
wardog,user-account--7c7b7450-5cfd-5683-a891-20644b37e384


We can also retrieve the machine names where this took place:

In [7]:
hosts = FIND x-oca-asset LINKED procs

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),artifact*,directory*,file*,process*,user-account*,windows-registry-key*,x-oca-asset*,x-oca-event*
hosts,x-oca-asset,1,60,60,66,95,64,4,24,59,60


In [8]:
DISP hosts

hostname,id
WORKSTATION5,x-oca-asset--40bf1119-f018-56a0-a6a5-9bfeab2bc1df


At this point, we have the basic evidence for creating an incident report.  